In [2]:
%matplotlib inline

import FPUT_Module as pranav

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import animation
from matplotlib.animation import FuncAnimation, ArtistAnimation

import scipy as sp
import scipy.constants as spc

from IPython.core.display import HTML
from IPython.display import display, Math, Latex

import os, time

pranav.customPlt()
pi = spc.pi

In [ ]:
N = 32
epsilon = 1
dt = 0.5
steps = 5000

eq = np.arange(N+2)
q0 = pranav.initMode(eq, 2)
p0 = 0*q0

t, q, p = pranav.fixed_fput(q0, p0, N, epsilon, dt, steps, method = 'leapfrog')
Q, P = pranav.dst(q), pranav.dst(p)
modeE = pranav.getEnergy(Q, P)

fig1 = plt.figure(figsize = (12,6), facecolor = 'white')
plt.plot(eq, q0, 'ko')
plt.xticks(np.arange(1,N+2,N-1))
plt.xlim(0, N+1)
plt.title('Initial Displacement Profile ' 
             + '(' + r'$N = {}$, '.format(N) + r'$\varepsilon = {}$'.format(epsilon) + ')')
plt.xlabel(r'Lattice Index $(n)$')
plt.ylabel(r'$\displaystyle \frac{\delta q_n(0)}{\ell}$', labelpad = 20, rotation = 0)
fig1.tight_layout()
plt.show()

fig2 = plt.figure(figsize = (12,6), facecolor = 'white')
plt.plot(t/dt, modeE)
plt.title('Exchange of Energy Between Normal Modes')
plt.xlabel('Algorithm Time Step', labelpad = 20)
plt.ylabel(r'$\mathrm{E}_k$', rotation = 0, labelpad=20)
fig2.tight_layout()
plt.show()

animateDiscrete = True
if animateDiscrete:
    display(Latex(r'$\mathrm{Creating}\, \mathrm{Animation}\ldots$'))
    fig = plt.figure(figsize = (12,6), facecolor = 'white')
    plt.xticks(np.arange(1,N+2,(N-1)/4))
    plt.xlim(0, N+1)
    plt.xlabel(r'$n$')
    plt.ylabel(r'$\displaystyle \frac{\delta q_n}{\ell}$', rotation = 0)
    plt.title(r'$\Delta \mathrm t = {:0.1f}$'.format(dt)
                  + r'$\quad n_\mathrm{{steps}}$'+r'$ = {}$'.format(steps))
    image = []
    for δq in q:
        image.append(plt.plot(eq, δq, 'k.'))
    image_animation = ArtistAnimation(fig, image, interval = 20, blit = True, repeat = True)
    plt.close()
    display(HTML(image_animation.to_jshtml()))

In [ ]:
L = 16.0
Nz = 128
dz = L/Nz 

dt = 5e-3
Nt = int(1e3)
T = Nt*dt

z  = np.arange(0, L, dz)
t  = np.arange(0, T, dt)
f0 = pranav.solitonIC(z,L/2,2)

delta = 0.01
stability = pranav.check_stability(dt,dz,delta)

f_o2 = pranav.kdv_solveFTCS(f0, z, dz, t, dt, delta, order = 2)
f_o6 = pranav.kdv_solveFTCS(f0, z, dz, t, dt, delta, order = 6)
f_lax = pranav.kdv_solveFTCS(f0, z, dz, t, dt, delta)

fig, [ax1,ax2,ax3] = plt.subplots(3,1, figsize=(12,15))
ax1.plot(z, f_o2.T, lw = 0.5, alpha = 0.01)
ax1.set_title(r'FTCS $2^{\mathrm{nd}}$ Order in $z$')

ax2.plot(z, f_o6.T, lw = 0.5, alpha = 0.01)
ax2.set_title(r'FTCS $6^{\mathrm{th}}$ Order in $z$')

ax3.plot(z, f_lax.T, lw = 0.5, alpha = 0.01)
ax3.set_title('Lax-Friedrichs')


for ax in [ax1,ax2,ax3]:
    ax.plot(z,f0,'g',lw = 2, alpha = 1)
    ax.set_ylim(0,np.max(f0)*3)
    ax.set_xlabel(r'$z$')
    ax.set_ylabel(r'$f(z)$', rotation = 0, labelpad = 20)
fig.suptitle('Comparison of Finite Difference Schemes for Soliton Initial Condition')
fig.tight_layout()
plt.show()